In [1]:
from vizproo import CustomWidget, MatrixLayout, MatrixCreator
import traitlets
import pandas as pd
import numpy as np

In [2]:
#d3.select(element).append(() => svg.node());

In [3]:
path = "../samples/leo.js"
csspath = "../samples/leo.css"
class CustomLeo(CustomWidget):
    _esm = CustomWidget.createWidgetFromLocalFile(paramList=[], filePath=path)
    _css = csspath

leo = CustomLeo()

In [65]:
import json
path_ = "../samples/states-albers-10m.json"
mapa_data = json.load(open(path_))

In [86]:
path = "../samples/mapa.js"
mapa_css = "../samples/mapa.css"
class CustomMapa(CustomWidget):
    _esm = CustomWidget.createWidgetFromLocalFile(paramList=["data"], filePath=path,
                                                  height = 610,
                                                  extra_imports=['import * as topojson from "https://esm.sh/topojson-client@3";'])
    _css = mapa_css
    data = traitlets.Dict(mapa_data).tag(sync=True)
    selectedValues = traitlets.List([]).tag(sync=True)

    def on_select_values(self, callback):
        self.observe(callback, names=["selectedValues"])
    

mapa = CustomMapa(data = mapa_data)

In [12]:
def read_csv_to_dicts(file_path: str) -> list[dict]:
    df = pd.read_csv(file_path)
    # Sustituye NaN, inf, -inf por None (null en JS)
    df = df.replace({np.nan: None, np.inf: None, -np.inf: None})
    return df.to_dict('records')
def get_columns(data:list[dict]) -> list[str]:
    return list(data[0].keys()) if data else []

In [13]:
penguins = read_csv_to_dicts("../samples/penguins.csv")

In [14]:
display(penguins[0])

{'species': 'Adelie',
 'island': 'Torgersen',
 'culmen_length_mm': 39.1,
 'culmen_depth_mm': 18.7,
 'flipper_length_mm': 181.0,
 'body_mass_g': 3750.0,
 'sex': 'MALE'}

In [17]:
path = "../samples/scatterplot_matrix.js"

class CustomScatter(CustomWidget):
    _esm = CustomWidget.createWidgetFromLocalFile(paramList=["data"], filePath=path)
    
    data = traitlets.List([]).tag(sync=True)

chart = CustomScatter(data=penguins)

In [82]:
m = MatrixCreator(7,4)
m

MatrixCreator(columns='4', rows='7')

In [87]:
layout = MatrixLayout(m.matrix,style='dark')
layout.add(leo,1)
layout.add(mapa,2)
layout

CustomLeo(elementId='lGTaAPQpTQ')

CustomMapa(data={'type': 'Topology', 'bbox': [-57.66491068874468, 12.97635452036684, 957.5235629133763, 606.56…

MatrixLayout(grid_areas=['lGTaAPQpTQ', 'gvMSEVNmwR'], grid_template_areas='\n"lGTaAPQpTQ lGTaAPQpTQ lGTaAPQpTQ…

In [88]:
mapa.selectedValues

[{'id': '48', 'name': 'Texas'}]